In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPRegressor

import chocolate as choco


C:\Users\rditlas8\AppData\Local\Continuum\anaconda3\envs\vrsense1\lib\site-packages\chocolate\mo\pyhv.py:16: ImportWarning: Using Python version of hypervolume module. Expect this to be slower.
  warnings.warn("Using Python version of hypervolume module. Expect this to be slower.", ImportWarning)


In [3]:
#load saved dataset rather than generating your own
dataset3_X = np.loadtxt("dataset3_X.csv",delimiter=',')
dataset3_Y = np.loadtxt("dataset3_Y.csv",delimiter=',')
print("generated X frequency bins with shape =",dataset3_X.shape)
print("generated Y (w1,w2,ratio_m1_over_m2) with shape =",dataset3_Y.shape)

generated X frequency bins with shape = (1000, 500)
generated Y (w1,w2,ratio_m1_over_m2) with shape = (1000, 10)


In [20]:
def build_MLPRegressor(params):    
    return MLPRegressor(hidden_layer_sizes=(int(params['hidden_layer1']),
                                           int(params['hidden_layer2']),
                                           # int(params['hidden_layer3']),
                                           ),
                                       activation='relu',
                                       solver='adam',
                                       learning_rate='adaptive',
                                       max_iter=int(params['max_iter']),
                                       learning_rate_init=params['learning_rate_init'],
                                       alpha=params['alpha'])

def _score(X, Y, params):
    #split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10,shuffle=True)
    
    #scale
    x_scaler_nl = StandardScaler()
    X_train_scaled_nl = x_scaler_nl.fit_transform(X_train)
    X_test_scaled_nl = x_scaler_nl.transform(X_test)

    y_scaler_nl = StandardScaler().fit(Y_train)
    Y_train_scaled_nl = y_scaler_nl.transform(Y_train)
    Y_test_scaled_nl = y_scaler_nl.transform(Y_test)
    
    #create ML model
    model = build_MLPRegressor(params)
    
    #train
    model.fit(X_train_scaled_nl, Y_train_scaled_nl)
    
    #predict
    Y_test_pred_scaled = model.predict(X_test_scaled_nl)
    
    #de-scale
    Y_test_pred_unscaled = y_scaler_nl.inverse_transform(Y_test_pred_scaled)

    # Chocolate minimizes the loss
    return mean_squared_error(Y_test_pred_unscaled, Y_test)


def main(X,Y,run):
    #1.15	2127	0.009	50
    # Connect to sqlite database in current directory
    conn = choco.SQLiteConnection(url="sqlite:///"+run+".db")
    s = {
      "hidden_layer1": choco.quantized_uniform(2,300,10), 
      "hidden_layer2":  choco.quantized_uniform(2,300,1),
      #"hidden_layer3":  choco.quantized_uniform(2,300,1), 
      "learning_rate_init": 0.005,#choco.quantized_uniform(0.005,0.005,0.0005),#choco.quantized_log(0,5,1,0.1), 
      "alpha": 1.0,#choco.quantized_uniform(1.0,1.0,0.1),#choco.quantized_log(0,5,1,0.1), 
      "max_iter": choco.quantized_uniform(50,500,50)
    }
    cv = choco.Repeat(repetitions=3, reduce=np.mean, rep_col="_repetition_id")
    sampler = choco.Bayes(conn, s,clear_db=False,crossvalidation=cv)
    for i in range(0,10):
        token, params = sampler.next()
        loss = _score(X, Y, params)
        #print("finished iteration",str(i),"loss",str(loss))
        sampler.update(token, loss)


In [5]:
from joblib import Parallel, delayed
Parallel(n_jobs=16)(delayed(main)(dataset3_Y,dataset3_X,"1layer_pend5_backwards_mlp_v1") for i in range(16))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [11]:
dataset3_X.mean()

0.017546220489433557

In [12]:
dataset3_Y.mean()

1.000705591427598

In [13]:
0.000112/dataset3_X.mean()

0.006383141034130233

In [21]:
from joblib import Parallel, delayed
Parallel(n_jobs=16)(delayed(main)(dataset3_X,dataset3_Y,"2layer_pend5_forwards_mlp_v1") for i in range(16))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]